In [1]:
#using pandas
import os
import pandas as pd
import numpy as np
from numpy import *

In [2]:
#os.chdir('C:\Users\srira\Desktop')
#os.chdir('C:\Users\caccess\Desktop')

In [25]:
#!type plan.csv

In [4]:
data = pd.read_table("plan.csv", sep=',', header=None)
data.head(10)

,0,1
0,Data Request Type,Wireless Account Type
1,WIRELESS,IR - Individual Regular
2,WIRELESS,IR - Individual Regular
3,WIRELESS,IR - Individual Regular
4,WIRELESS,IR - Individual Regular
5,WIRELESS,IR - Individual Regular
6,WIRELESS,IR - Individual Regular
7,WIRELESS,IR - Individual Regular
8,WIRELESS,IR - Individual Regular
9,WIRELESS,IR - Individual Regular


In [5]:
data = genfromtxt('plan.csv',skip_header=1, delimiter=',', dtype=None,usecols=arange(0,2))       

In [6]:
x=data

In [7]:
a=np.array(x)

In [8]:
from numpy import *

# Analyzing ATT data sets with Apriori algorithm

In [9]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

In [10]:
C1 = createC1(a)
C1

[frozenset({'CB-U-VERSE'}),
 frozenset({'CB-WIRELINE'}),
 frozenset({'IR - Individual Regular'}),
 frozenset({'Unified (Uverse + Wireless + DTV)'}),
 frozenset({'WIRELESS'})]

In [11]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [12]:
D = list(map(set,a))

In [13]:
L1,suppDat0 = scanD(D,C1,0.01)
L1

[frozenset({'WIRELESS'}),
 frozenset({'CB-WIRELINE'}),
 frozenset({'IR - Individual Regular'}),
 frozenset({'Unified (Uverse + Wireless + DTV)'}),
 frozenset({'CB-U-VERSE'})]

In [14]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [15]:
def apriori(dataSet, minSupport = 0.1):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [16]:
L,suppData = apriori(a)
L

[[frozenset({'WIRELESS'}), frozenset({'IR - Individual Regular'})],
 [frozenset({'IR - Individual Regular', 'WIRELESS'})],
 []]

In [17]:
L[0]

[frozenset({'WIRELESS'}), frozenset({'IR - Individual Regular'})]

In [18]:
L[1]

[frozenset({'IR - Individual Regular', 'WIRELESS'})]

In [19]:
aprioriGen(L[0],2)

[frozenset({'IR - Individual Regular', 'WIRELESS'})]

# Mining association rules from frequent item sets

In [20]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

In [21]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [22]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [23]:
L,suppData= apriori(a,minSupport=0.1)

In [24]:
rules= generateRules(L,suppData, minConf=0.1)

(frozenset(['IR - Individual Regular']), '-->', frozenset(['WIRELESS']), 'conf:', 0.8334489937543372)
(frozenset(['WIRELESS']), '-->', frozenset(['IR - Individual Regular']), 'conf:', 1.0)


I have taken only a small part of the dataset for which I have analyzed the above results.

The above rules confidently says that in NEWYORK city wirless is more and there is more confidence to install more wireless  over there and vice versa.